In [1]:
using Printf, CSV, DataFrames, Statistics, Glob, XLSX

In [2]:
directory = joinpath(homedir(), "research", "GADS", "Events")
files = glob("*.csv", directory)
x = Dict()
i = 1
for f in files
    df = DataFrame(CSV.File(f, select=["EventStartDT", "EventEndDT", "UnitID", "UnitTypeCode", "EventTypeCode", "OutageContinuationFlag"]))
    x[i] = df
    i +=1
end
merged = reduce(vcat, values(x))
sort!(merged, :EventStartDT)

Row,EventEndDT,EventStartDT,EventTypeCode,OutageContinuationFlag,UnitID,UnitTypeCode
,String31,DateTime,String3,Int64,Int64,Int64
1,2013-01-05T06:54:00.0,2013-01-01T04:00:00,RS,2,5408,100
2,2013-01-02T22:16:00.0,2013-01-01T04:00:00,RS,2,5423,500
3,2013-01-02T22:21:00.0,2013-01-01T04:00:00,RS,2,5424,500
4,2013-01-12T22:05:00.0,2013-01-01T04:00:00,RS,2,5427,500
5,2013-01-26T19:10:00.0,2013-01-01T04:00:00,RS,2,5415,300
6,2013-01-28T17:16:00.0,2013-01-01T04:00:00,RS,2,11240,300
7,2013-01-02T20:57:00.0,2013-01-01T04:00:00,RS,2,5426,500
8,2013-02-01T04:00:00.0,2013-01-01T04:00:00,RS,2,5420,300
9,2013-01-01T18:52:00.0,2013-01-01T04:00:00,RS,2,5407,100


In [3]:
fname = joinpath(homedir(), "research", "GADS", "GADS_Unit_2023_20240422.xlsx")
units = DataFrame(XLSX.readtable(fname, "Sheet1"));

In [4]:
# event_gen_ids = unique(merged[!, "UnitID"])
# all_gen_ids = unique(units[!, "UnitID"])

# not_in_event_gen_ids = setdiff(all_gen_ids, event_gen_ids)

# units[units.UnitID in not_in_event_gen_ids, !]

# not_in_events_df = filter(row -> row[:UnitID] in not_in_event_gen_ids, units)

# a = not_in_events_df[!, :UnitRetireDT_grp]
# count(x -> true, skipmissing(a))

In [9]:
units

Row,UnitID,AliasID,CommercialDT_grp,ID_SK,NERCID_AliasID,RatingMW_grp,RegionCode,StateName,UnitRetireDT_grp,UnitTimeZoneCode,UnitTypeCode,UnitTypeCodeDesc,RatingMW_BrochureGroup
,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any
1,5050,0x97B421A0918CC799265B9E89118BA60B,1,12047,0xE8E5E322646F435388EEEE255A077D67,3,TRE,Texas,Retired,CPT,100,Fossil-Steam,400-599
2,5052,0x4F45850F285D33C649CB2660E311C785,1,12049,0xE8E5E322646F435388EEEE255A077D67,3,TRE,Texas,Retired,CPT,100,Fossil-Steam,800+
3,11835,0xC5AC4AB466AF63C98BB7521CE68A62DC,5,12186,0xE45673082C83B4124005C54A93ABC8E5,3,SERC,Mississippi,missing,CPT,850,Combined Cycle Block,All
4,4773,0x8C5A80F096D4D3C55EAEB512D0FD0CDC,3,12195,0xCF0E398F6F1BAC73B41A0CB6B3967BE5,2,SERC,Kentucky,missing,EST,300,Gas Turbine/Jet Engine (Simple Cycle Operation),50+
5,12295,0x281D8C92796753B548725105E2037D59,2,12328,0xE42F8966B0CDB3D578D436CFDA61B29A,1,WECC,California,missing,PST,650,Fluidized Bed,All
6,12264,0xB67AA6AAB6FCE01DA1762229B6482EB2,5,12483,0xD3E3BDBE360C63F3E927FFE438357FAC,3,SERC,Kentucky,missing,EPT,852,CC steam units,All
7,287,0x0FF9D5C271BF9EA95253A94AB2C31681,1,12506,0x0C4CBDEFD4B1D0EFCF8B0B9495621A14,1,MRO,Iowa,Retired,CPT,100,Fossil-Steam,<100
8,5286,0x66324F9FCEC968D0B1E5CD2BB8C1F32C,1,12564,0xB135DAB6216C01F3B62F533A5FBACDE5,1,MRO,South Dakota,missing,CPT,300,Gas Turbine/Jet Engine (Simple Cycle Operation),50+
9,1096,0x6AB07F384BDD639EDE65CCEAE98F2EDD,4,12740,0xACA7783C349A082F7B7AE801309E8A87,1,WECC,Wyoming,missing,MPT,300,Gas Turbine/Jet Engine (Simple Cycle Operation),<50


In [10]:
gen_ids = unique(merged[!, "UnitID"])
merged.CommercialDT_grp = missings(Int, nrow(merged))
merged.RatingMW_grp = missings(Int, nrow(merged))
merged.StateName = missings(String, nrow(merged))

myDict = Dict()

for u in gen_ids

    my_unit = filter(:UnitID => ==(u), units)
    subset = @view merged[(merged.UnitID .== u), :]

    commercial_dt_grp = my_unit[1, :CommercialDT_grp]
    subset.CommercialDT_grp .= commercial_dt_grp

    rating = my_unit[1, :RatingMW_grp]
    subset.RatingMW_grp .= rating

    state = my_unit[1, :StateName]
    subset.StateName .= state 

    unit_type_code = subset[1, :UnitTypeCode]

    # val = units[units.UnitID .== u, :CommercialDT_grp][1]
    # vec = merged[merged.UnitID .== u, :CommercialDT_grp]
    # new_vec = fill(val, length(vec))
    # merged[merged.UnitID .== u, :CommercialDT_grp] = new_vec

    # val = units[units.UnitID .== u, :RatingMW_grp][1]
    # vec = merged[merged.UnitID .== u, :RatingMW_grp]
    # new_vec = fill(val, length(vec))
    # merged[merged.UnitID .== u, :RatingMW_grp] = new_vec

    # val = units[units.UnitID .== u, :StateName][1]
    # vec = merged[merged.UnitID .== u, :StateName]
    # new_vec = fill(val, length(vec))
    # merged[merged.UnitID .== u, :StateName] = new_vec

    fname = @sprintf "gen_%d_type_%d_dtgrp_%d_rating_%d_state_%s_raw.csv" u unit_type_code commercial_dt_grp rating state
    path = joinpath(homedir(), "research", "GADS_Processed", "raw_data", fname)
    CSV.write(path, subset)

end